In [10]:
import pandas as pd
import numpy as np
from datetime import time, datetime
import gradio as gr
import re

In [11]:
# 上午打卡时间：8:00之前、12:00之后；
# 下午打卡时间：13:00之前、15:30之后
class AttendanceConfig:
    MORNING_START = time(8, 0)      # 上午上班时间
    MORNING_END = time(12, 0)       # 上午下班时间
    AFTERNOON_START = time(13, 00)  # 下午上班时间
    AFTERNOON_END = time(15, 30)    # 下午下班时间
    ALLOWED_LATE_MINUTES = 0       # 允许迟到分钟数
    ALLOWED_EARLY_MINUTES = 0      # 允许早退分钟数

In [12]:
def time_add_minutes(t, minutes):
    """时间加减分钟"""
    if not isinstance(t, time):
        return None
    total = t.hour * 60 + t.minute + minutes
    return time(total // 60 % 24, total % 60)

def process_attendance_data(df):
    """处理考勤数据主逻辑"""
    # 提取日期行（假设第3行是日期行）
    date_row = df.iloc[0, 2:].values

    results = []
    for row_idx in range(1, len(df)):
        row = df.iloc[row_idx]
        employee = row[0]
        department = row[1]
        
        for col_idx in range(2, min(len(row), len(date_row)+2)):
            date_str = date_row[col_idx-2]
            print("data_str:",data_str)
            date = f"2025-06-{date_str}" if str(date_str).isdigit() else f"特殊日期_{col_idx}"
            
            # 处理时间数据
            times = process_cell_times(row[col_idx])
            periods = assign_time_periods(times)
            status = check_attendance_status(periods)
            
            record = {
                'employee': employee,
                'department': department,
                'date': date,
                **periods,
                **status
            }
            results.append(record)

    return pd.DataFrame(results)

def parse_time(time_str):
    """将时间字符串转换为time对象"""
    if pd.isna(time_str) or not isinstance(time_str, str):
        return None
    
    try:
        # 清理字符串并提取时间部分
        time_str = re.sub(r'[^\d:]', '', time_str.strip())
        if ':' in time_str:
            parts = time_str.split(':')
            if len(parts) >= 2:
                return time(int(parts[0]), int(parts[1]))
        elif len(time_str) == 4 and time_str.isdigit():
            return time(int(time_str[:2]), int(time_str[2:]))
    except:
        return None
    return None

def process_cell_times(cell_value):
    """处理单元格内的多个时间记录"""
    if pd.isna(cell_value) or cell_value == '':
        return []
    
    # 统一处理换行符和空格
    if isinstance(cell_value, str):
        times = []
        # 先尝试按换行符分割
        for t in cell_value.split('\n'):
            t = t.strip()
            if t:
                # 再尝试按空格分割（处理"07:35 12:08"格式）
                for subt in t.split():
                    parsed = parse_time(subt)
                    if parsed:
                        times.append(parsed)
        return times
    return []

def assign_time_periods(times):
    """
    智能分配时间点到对应时间段（枚举所有情况）
        上午打卡时间：8:00之前、12:00之后；
        下午打卡时间：13:00之前、15:30之后
    """
    config = AttendanceConfig()
    # 过滤无效时间并按时间排序
    valid_times = sorted([t for t in times if isinstance(t, time)])
    print("valid_times:", valid_times)
    num_times = len(valid_times)
    
    # 初始化结果
    result = {
        'morning_start': None,
        'morning_end': None,
        'afternoon_start': None,
        'afternoon_end': None
    }
    
    # 根据时间点数量处理不同情况
    if num_times == 0:
        # 无有效时间点，全部缺卡
        return result
    # MORNING_START 
    elif num_times == 1:
        # 只有一个时间点
        t = valid_times[0]
        if t <= config.MORNING_START:
            result['morning_start'] = t
        elif config.MORNING_END <= t <= config.AFTERNOON_START:
            # 可能在上午下班和下午上班之间
            result['morning_end'] = t
        elif config.AFTERNOON_START < t < config.AFTERNOON_END:
            result['afternoon_start'] = t
        elif t >= config.AFTERNOON_END:
            result['afternoon_end'] = t
        else:
            # 无法确定时间段，按时间顺序分配
            result['morning_start'] = t
    
    elif num_times == 2:
        t1, t2 = valid_times
        print("num_times == 2:",t1, t2)
        print(config.MORNING_START, config.MORNING_END, config.AFTERNOON_START, config.AFTERNOON_END)
        # 可能的情况：上午两次打卡、下午两次打卡、跨午休打卡
        if t1 <= config.MORNING_START and t2 >= config.MORNING_END:
            # 上午两次打卡
            result['morning_start'] = t1
            result['morning_end'] = t2
        elif t1 <= config.AFTERNOON_START and t2 >= config.AFTERNOON_END:
            # 下午两次打卡
            result['afternoon_start'] = t1
            result['afternoon_end'] = t2
        elif t1 < config.MORNING_END and t2 > config.AFTERNOON_START:
            # 跨午休打卡
            result['morning_end'] = t1
            result['afternoon_start'] = t2
        else:
            # 默认分配
            result['morning_start'] = t1
            result['morning_end'] = t2
    
    elif num_times == 3:
        t1, t2, t3 = valid_times
        # 可能缺上午上班、上午下班、下午上班或下午下班
        if t1 <= config.MORNING_START and t2 >= config.MORNING_END and t3 >= config.AFTERNOON_END:
            # 缺下午上班
            result['morning_start'] = t1
            result['morning_end'] = t2
            result['afternoon_end'] = t3
        elif t1 <= config.MORNING_START and t2 <= config.AFTERNOON_START and t3 >= config.AFTERNOON_END:
            # 缺上午下班
            result['morning_start'] = t1
            result['afternoon_start'] = t2
            result['afternoon_end'] = t3
        elif t1 >= config.MORNING_END and t2 <= config.AFTERNOON_START and t3 >= config.AFTERNOON_END:
            # 缺上午上班
            result['morning_end'] = t1
            result['afternoon_start'] = t2
            result['afternoon_end'] = t3
        elif t1 <= config.MORNING_START and t2 >= config.MORNING_END and t3 <= config.AFTERNOON_START:
            # 缺下午下班
            result['morning_start'] = t1
            result['morning_end'] = t2
            result['afternoon_start'] = t3
        else:
            # 默认分配
            result['morning_start'] = t1
            result['morning_end'] = t2
            result['afternoon_start'] = t3
    
    elif num_times >= 4:
        # 有4个或更多时间点，取前4个
        t1, t2, t3, t4 = valid_times[:4]
        result['morning_start'] = t1
        result['morning_end'] = t2
        result['afternoon_start'] = t3
        result['afternoon_end'] = t4
    
    return result

def check_attendance_status(record):
    """检查考勤状态（优化版）"""
    config = AttendanceConfig()
    status = {
        'late': False,
        'early_leave': False,
        'missing': False,
        'status_details': []
    }
    
    # 检查上午上班
    if record['morning_start']:
        if record['morning_start'] > time_add_minutes(config.MORNING_START, config.ALLOWED_LATE_MINUTES):
            status['late'] = True
            status['status_details'].append('上午迟到')
    else:
        status['missing'] = True
        status['status_details'].append('上午上班缺卡')
    
    # 检查上午下班
    if record['morning_end']:
        if record['morning_end'] < time_add_minutes(config.MORNING_END, -config.ALLOWED_EARLY_MINUTES):
            status['early_leave'] = True
            status['status_details'].append('上午早退')
    else:
        status['missing'] = True
        status['status_details'].append('上午下班缺卡')
    
    # 检查下午上班
    if record['afternoon_start']:
        if record['afternoon_start'] > time_add_minutes(config.AFTERNOON_START, config.ALLOWED_LATE_MINUTES):
            status['late'] = True
            status['status_details'].append('下午迟到')
    else:
        status['missing'] = True
        status['status_details'].append('下午上班缺卡')
    
    # 检查下午下班
    if record['afternoon_end']:
        if record['afternoon_end'] < time_add_minutes(config.AFTERNOON_END, -config.ALLOWED_EARLY_MINUTES):
            status['early_leave'] = True
            status['status_details'].append('下午早退')
    else:
        status['missing'] = True
        status['status_details'].append('下午下班缺卡')
    
    return status

In [13]:
##给df的前两列加上列名：“姓名”，“部门”
input_file = "./工作簿2.xlsx"
df = pd.read_excel(input_file, header=3) #, columns=["姓名", "部门"]skiprows=3,
print(df.columns)
df = df.rename(columns={'Unnamed: 0': '姓名', 'Unnamed: 1': '部门'})
# print("df.head(2):",df.head(2))
df.head(7)

Index(['Unnamed: 0', 'Unnamed: 1', '日', '2', '3', '4', '5', '6', '六', '日.1',
       '9', '10', '11', '12', '13', '六.1', '日.2', '16', '17', '18', '19', '20',
       '六.2', '日.3', '23', '24', '25', '26', '27', '六.3', '日.4', '30'],
      dtype='object')


,姓名,部门,日,2,3,4,5,6,六,日.1,...,六.2,日.3,23,24,25,26,27,六.3,日.4,30
0,刘旸,财务部,NaN,07:35 \n12:08 \n12:31 \n17:31,07:35 \n12:12 \n12:32 \n17:33,07:35 \n12:11 \n12:31 \n17:33,07:40 \n12:11 \n12:31 \n17:32,07:32 \n12:11 \n12:32 \n17:32,07:38 \n12:12 \n12:32 \n17:33,NaN,...,07:27 \n12:01 \n12:32 \n17:34,NaN,07:27 \n12:04 \n12:31 \n17:33,07:31 \n12:05 \n12:31 \n17:34,07:31 \n12:04 \n12:32 \n17:34,07:37 \n12:03 \n12:31 \n17:32,07:35 \n12:03 \n12:31 \n17:32,07:29 \n12:01 \n12:31 \n17:33,NaN,07:28 \n12:03 \n12:31 \n17:32
1,沈宝玉,财务部,NaN,07:50 \n12:14 \n12:31 \n17:37,07:48 \n12:13 \n12:31 \n17:41,07:50 \n12:11 \n12:31 \n17:42,07:54 \n12:11 \n12:32 \n17:41,07:55 \n12:10 \n12:32 \n17:36,07:53 \n12:12 \n12:32 \n17:38,NaN,...,07:54 \n12:04 \n12:32 \n17:44,NaN,07:41 \n12:04 \n12:32 \n17:35,07:55 \n12:05 \n12:31 \n17:37,07:54 \n12:03 \n12:32 \n17:32,07:55 \n12:03 \n12:31 \n17:38,07:52 \n12:03 \n12:32 \n17:38,07:56 \n12:02 \n12:32 \n17:40,NaN,07:55 \n12:03 \n12:32 \n17:39
2,施军,财务部,NaN,07:48 \n17:38,13:42 \n17:33,13:44 \n17:33,09:53 \n12:02 \n13:27 \n17:34,14:31 \n17:32,13:35 \n17:36,NaN,...,14:27 \n17:31,NaN,13:39 \n17:33,13:30 \n17:32,12:52 \n17:34,13:45 \n17:31,13:35 \n17:33,13:35 \n17:37,NaN,14:16 \n17:32
3,谭益云,财务部,NaN,07:58 \n12:00 \n13:29 \n17:37,08:01 \n12:10 \n13:28 \n17:33,07:55 \n12:17 \n13:29 \n17:36,07:59 \n12:02 \n13:28 \n17:31,07:55 \n12:29 \n12:30 \n17:32,08:03 \n12:00 \n12:32 \n17:40,NaN,...,07:58 \n12:00 \n13:26 \n17:32,NaN,07:59 \n12:02 \n13:29 \n17:44,13:25 \n17:32,07:59 \n12:02 \n13:28 \n17:34,07:52 \n12:23 \n13:09 \n17:32,08:00 \n12:00 \n12:38 \n17:30,07:57 \n12:00 \n12:31 \n17:36,NaN,07:55 \n12:29 \n12:30 \n17:51
4,袁政,财务部,NaN,07:46 \n12:10 \n13:26 \n18:13,07:58 \n12:15 \n13:25 \n17:39,07:54 \n12:01 \n13:26 \n17:38,NaN,07:56 \n12:06 \n13:25 \n17:33,07:47 \n12:00 \n13:24 \n17:42,NaN,...,07:49 \n12:00 \n13:25 \n18:07,NaN,07:49 \n12:04 \n13:25 \n19:06,07:54 \n12:09 \n13:24 \n17:44,07:59 \n12:02 \n13:25 \n18:01,07:44 \n12:03 \n13:26 \n18:44,07:55 \n12:03 \n13:26 \n17:57,07:50 \n12:01 \n13:25 \n17:47,NaN,07:46 \n12:03 \n13:25 \n17:33
5,刘军,采购部,NaN,07:51 \n12:14 \n12:32 \n17:48,07:54 \n12:19 \n12:30 \n17:40,07:55 \n12:23 \n12:32 \n18:10,07:54 \n12:19 \n12:30 \n17:52,07:40 \n12:18 \n12:30 \n18:07,07:36 \n12:23 \n12:30 \n17:49,NaN,...,07:36 \n12:04 \n12:32 \n18:09,NaN,07:56 \n12:08 \n12:32 \n17:57,07:48 \n12:06 \n12:31 \n17:58,07:52 \n12:08 \n12:31 \n17:41,07:49 \n12:06 \n12:35 \n17:54,07:53 \n12:09 \n12:34 \n17:46,07:32 \n12:27 \n12:38 \n17:50,NaN,07:51 \n12:09 \n12:36 \n17:55
6,刘学义,采购部,07:45 \n11:00,07:39 \n12:05 \n13:26 \n17:34,07:44 \n12:00 \n13:31 \n17:35,07:48 \n12:01 \n13:30 \n17:34,07:51 \n12:09 \n13:26 \n17:32,07:44 \n12:05 \n12:34 \n17:35,NaN,07:48 \n12:24 \n13:31 \n17:35,...,NaN,07:57 \n12:01 \n13:28 \n17:01,07:54 \n12:03 \n12:40 \n13:05 \n17:37,07:50 \n12:12 \n12:44 \n17:39,07:52 \n12:17 \n17:41,07:51 \n12:09 \n12:40 \n17:35,07:52 \n12:03 \n12:45 \n17:32,07:55 \n12:12 \n12:35 \n17:43,06:42,07:53 \n12:02 \n12:44 \n17:32


In [14]:
df.columns.values

array(['姓名', '部门', '日', '2', '3', '4', '5', '6', '六', '日.1', '9', '10',
       '11', '12', '13', '六.1', '日.2', '16', '17', '18', '19', '20',
       '六.2', '日.3', '23', '24', '25', '26', '27', '六.3', '日.4', '30'],
      dtype=object)

In [15]:
df
# 提取日期行（假设第3行是日期行）
date_row = df.columns.values[2:]#df.iloc[0, 2:].values
date_row

array(['日', '2', '3', '4', '5', '6', '六', '日.1', '9', '10', '11', '12',
       '13', '六.1', '日.2', '16', '17', '18', '19', '20', '六.2', '日.3',
       '23', '24', '25', '26', '27', '六.3', '日.4', '30'], dtype=object)

In [16]:
results = []
for row_idx in range(0, len(df)):
    row = df.iloc[row_idx]
    employee = row[0]
    department = row[1]
    
    for col_idx in range(2, min(len(row), len(date_row)+2)):
        date_str = date_row[col_idx-2]
        print("date_str:",date_str)
        date = f"2025-06-{date_str}" if str(date_str).isdigit() else f"特殊日期_{col_idx}" ###需要去掉
        print("date:",date)
        
        # 处理时间数据
        times = process_cell_times(row[col_idx])
        print("times:",times)
        periods = assign_time_periods(times)
        print("periods:",periods)
        status = check_attendance_status(periods)
        print("status:",status)
        
        record = {
            'employee': employee,
            'department': department,
            'date': date,
            **periods,
            **status
        }
        print("record:",record)
        results.append(record)
        print()

date_str: 日
date: 特殊日期_2
times: []
valid_times: []
periods: {'morning_start': None, 'morning_end': None, 'afternoon_start': None, 'afternoon_end': None}
status: {'late': False, 'early_leave': False, 'missing': True, 'status_details': ['上午上班缺卡', '上午下班缺卡', '下午上班缺卡', '下午下班缺卡']}
record: {'employee': '刘旸', 'department': '财务部', 'date': '特殊日期_2', 'morning_start': None, 'morning_end': None, 'afternoon_start': None, 'afternoon_end': None, 'late': False, 'early_leave': False, 'missing': True, 'status_details': ['上午上班缺卡', '上午下班缺卡', '下午上班缺卡', '下午下班缺卡']}

date_str: 2
date: 2025-06-2
times: [datetime.time(7, 35), datetime.time(12, 8), datetime.time(12, 31), datetime.time(17, 31)]
valid_times: [datetime.time(7, 35), datetime.time(12, 8), datetime.time(12, 31), datetime.time(17, 31)]
periods: {'morning_start': datetime.time(7, 35), 'morning_end': datetime.time(12, 8), 'afternoon_start': datetime.time(12, 31), 'afternoon_end': datetime.time(17, 31)}
status: {'late': False, 'early_leave': False, 'missin

 [datetime.time(17, 33)]
valid_times: [datetime.time(17, 33)]
periods: {'morning_start': None, 'morning_end': None, 'afternoon_start': None, 'afternoon_end': datetime.time(17, 33)}
status: {'late': False, 'early_leave': False, 'missing': True, 'status_details': ['上午上班缺卡', '上午下班缺卡', '下午上班缺卡']}
record: {'employee': '吕德江', 'department': '企业管理部', 'date': '2025-06-13', 'morning_start': None, 'morning_end': None, 'afternoon_start': None, 'afternoon_end': datetime.time(17, 33), 'late': False, 'early_leave': False, 'missing': True, 'status_details': ['上午上班缺卡', '上午下班缺卡', '下午上班缺卡']}

date_str: 六.1
date: 特殊日期_15
times: [datetime.time(7, 19)]
valid_times: [datetime.time(7, 19)]
periods: {'morning_start': datetime.time(7, 19), 'morning_end': None, 'afternoon_start': None, 'afternoon_end': None}
status: {'late': False, 'early_leave': False, 'missing': True, 'status_details': ['上午下班缺卡', '下午上班缺卡', '下午下班缺卡']}
record: {'employee': '吕德江', 'department': '企业管理部', 'date': '特殊日期_15', 'morning_start': datetim

In [19]:
result_df = pd.DataFrame(results)

In [21]:
detailed_report = result_df.rename(columns={
    'employee': '员工姓名',
    'date': '日期',
    'morning_start': '上午上班',
    'morning_end': '上午下班',
    'afternoon_start': '下午上班',
    'afternoon_end': '下午下班',
    'late': '迟到',
    'early_leave': '早退',
    'missing': '缺卡'
})
detailed_report

,员工姓名,department,日期,上午上班,上午下班,下午上班,下午下班,迟到,早退,缺卡,status_details
0,刘旸,财务部,特殊日期_2,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
1,刘旸,财务部,2025-06-2,07:35:00,12:08:00,12:31:00,17:31:00,False,False,False,[]
2,刘旸,财务部,2025-06-3,07:35:00,12:12:00,12:32:00,17:33:00,False,False,False,[]
3,刘旸,财务部,2025-06-4,07:35:00,12:11:00,12:31:00,17:33:00,False,False,False,[]
4,刘旸,财务部,2025-06-5,07:40:00,12:11:00,12:31:00,17:32:00,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...
775,卫良金,质量与工艺部-工艺处,2025-06-26,07:53:00,12:13:00,12:42:00,20:32:00,False,False,False,[]
776,卫良金,质量与工艺部-工艺处,2025-06-27,07:54:00,12:12:00,12:37:00,17:50:00,False,False,False,[]
777,卫良金,质量与工艺部-工艺处,特殊日期_29,07:59:00,12:06:00,12:31:00,17:32:00,False,False,False,[]
778,卫良金,质量与工艺部-工艺处,特殊日期_30,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"


In [22]:
detailed_report= detailed_report[detailed_report['status_details'].apply(len) > 0]
detailed_report

,员工姓名,department,日期,上午上班,上午下班,下午上班,下午下班,迟到,早退,缺卡,status_details
0,刘旸,财务部,特殊日期_2,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
7,刘旸,财务部,特殊日期_9,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
14,刘旸,财务部,特殊日期_16,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
21,刘旸,财务部,特殊日期_23,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
28,刘旸,财务部,特殊日期_30,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
...,...,...,...,...,...,...,...,...,...,...,...
757,卫良金,质量与工艺部-工艺处,特殊日期_9,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
764,卫良金,质量与工艺部-工艺处,特殊日期_16,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
770,卫良金,质量与工艺部-工艺处,特殊日期_22,07:54:00,12:18:00,13:26:00,17:53:00,True,False,False,[下午迟到]
771,卫良金,质量与工艺部-工艺处,特殊日期_23,None,None,None,None,False,False,True,"[上午上班缺卡, 上午下班缺卡, 下午上班缺卡, 下午下班缺卡]"
